<a href="https://colab.research.google.com/github/harshitadd/Celestini/blob/master/Generalised_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install firebase-admin
import os 

In [0]:
#Get Images from Firebase Storage
import datetime
import firebase_admin
from firebase_admin import credentials
from firebase_admin import storage

# Fetch the service account key JSON file contents
cred = credentials.Certificate("credentials.json")

# Initialize the app with a service account, granting admin privileges
app = firebase_admin.initialize_app(cred, {
    'storageBucket': 'fir-4ca2c.appspot.com',
}, name='storage')

In [28]:
#Loading E1 - Lenovo data : 504 -- sscb labels 

data = []
reading = []
t = 0
bucket = storage.bucket(app=app)
blobs = bucket.list_blobs()

imglenovo = []

mits = set()
for blob in blobs:
  if blob.name[7:16]=="lenovoinS" :
    if int(blob.name[-8:-6])<19 and int(blob.name[-8:-6])>5:
      blob.download_to_filename('/content/' + "lenovo" + str(blob.name.split('/')[-1])+ '.jpeg')
      imglenovo.append("lenovo" + str(blob.name.split('/')[-1])+ '.jpeg') 
      print("lenovo" + str(blob.name.split('/')[-1])+ '.jpeg')



lenovo2019-06-27 17:01:04.jpeg
lenovo2019-06-27 17:05:48.jpeg
lenovo2019-06-27 17:10:27.jpeg
lenovo2019-06-27 17:15:58.jpeg
lenovo2019-06-27 18:04:29.jpeg
lenovo2019-06-27 18:08:53.jpeg
lenovo2019-06-27 18:17:05.jpeg
lenovo2019-06-27 18:19:07.jpeg
lenovo2019-06-27 18:26:07.jpeg
lenovo2019-06-27 18:28:53.jpeg
lenovo2019-06-27 18:35:27.jpeg
lenovo2019-06-27 18:39:31.jpeg
lenovo2019-06-27 18:47:06.jpeg
lenovo2019-06-27 18:49:07.jpeg
lenovo2019-06-27 18:56:06.jpeg
lenovo2019-06-28 06:00:07.jpeg
lenovo2019-06-28 06:04:07.jpeg
lenovo2019-06-28 06:09:28.jpeg
lenovo2019-06-28 06:14:37.jpeg
lenovo2019-06-28 06:18:53.jpeg
lenovo2019-06-28 06:25:28.jpeg
lenovo2019-06-28 06:29:29.jpeg
lenovo2019-06-28 06:35:27.jpeg
lenovo2019-06-28 06:38:53.jpeg
lenovo2019-06-28 06:44:28.jpeg
lenovo2019-06-28 06:49:29.jpeg
lenovo2019-06-28 06:56:37.jpeg
lenovo2019-06-28 07:00:28.jpeg
lenovo2019-06-28 07:04:07.jpeg
lenovo2019-06-28 07:09:28.jpeg
lenovo2019-06-28 07:13:53.jpeg
lenovo2019-06-28 07:18:53.jpeg
lenovo20

In [29]:
##Loading E2 - Mi data : 760 -- sscb labels 
bucket = storage.bucket(app=app)
blobs = bucket.list_blobs()

imgminote = []

mits = set()
for blob in blobs:
  if blob.name[7:16]=="minoteSSC" :
    if int(blob.name[-8:-6])<19 and int(blob.name[-8:-6])>5:
        print(blob.name)

        blob.download_to_filename('/content/' + "minote" + str(blob.name.split('/')[-1])+ '.jpeg')
        imgminote.append("minote" + str(blob.name.split('/')[-1])+ '.jpeg') 
        print("minote" + str(blob.name.split('/')[-1])+ '.jpeg')


images/minoteSSCB.JPG/2019-06-27 15:14:35
minote2019-06-27 15:14:35.jpeg
images/minoteSSCB.JPG/2019-06-27 15:15:34
minote2019-06-27 15:15:34.jpeg
images/minoteSSCB.JPG/2019-06-27 15:16:22
minote2019-06-27 15:16:22.jpeg
images/minoteSSCB.JPG/2019-06-27 15:16:40
minote2019-06-27 15:16:40.jpeg
images/minoteSSCB.JPG/2019-06-27 15:16:57
minote2019-06-27 15:16:57.jpeg
images/minoteSSCB.JPG/2019-06-27 15:19:21
minote2019-06-27 15:19:21.jpeg
images/minoteSSCB.JPG/2019-06-27 15:24:14
minote2019-06-27 15:24:14.jpeg
images/minoteSSCB.JPG/2019-06-27 16:42:56
minote2019-06-27 16:42:56.jpeg
images/minoteSSCB.JPG/2019-06-27 16:46:16
minote2019-06-27 16:46:16.jpeg
images/minoteSSCB.JPG/2019-06-27 16:49:53
minote2019-06-27 16:49:53.jpeg
images/minoteSSCB.JPG/2019-06-27 16:50:47
minote2019-06-27 16:50:47.jpeg
images/minoteSSCB.JPG/2019-06-27 16:51:15
minote2019-06-27 16:51:15.jpeg
images/minoteSSCB.JPG/2019-06-27 16:54:37
minote2019-06-27 16:54:37.jpeg
images/minoteSSCB.JPG/2019-06-27 16:59:33
minote201

In [46]:
timestamps=[]
for img in imglenovo:
  timestamps.append(img[-24:])
  
print(len(timestamps))
for img in imgminote:
  timestamps.append(img[-24:])
  
print(len(timestamps))

505
1293


In [0]:
def transmission(img, lt = 230, retMean = False):
  ker = np.ones((3, 3))/9.0
  imgrgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  b, g, r = cv2.split(img)
  hsv = cv2.cvtColor(imgrgb, cv2.COLOR_RGB2HSV)
  _, _, v = cv2.split(hsv)
  v_blur = cv2.filter2D(v, -1, ker)
  _,building_mask = cv2.threshold(v_blur, lt, 255, cv2.THRESH_BINARY)
  sky = cv2.bitwise_and(imgrgb, imgrgb, mask = building_mask)
  sky_mask = cv2.bitwise_not(building_mask, building_mask.copy())
  building = cv2.bitwise_and(imgrgb, imgrgb, mask = sky_mask)
 
  al = max(v_blur.flatten())
  #Airlight Found. Now calculate Transmission Map
  img_norm = img/float(al)
  b, g, r = cv2.split(img_norm)
  kernel = np.ones((3,3),np.uint8)
  im_new = np.zeros((img.shape[0], img.shape[1]))
  for i in range(img.shape[0]):
    for j in range(img.shape[1]):
      im_new[i][j] = min(b[i][j], g[i][j], r[i][j])
  dcp = cv2.erode(im_new, kernel, iterations = 1)
  #ax[3].imshow(im_new, cmap = 'gray')
  #ax[4].imshow(dcp, cmap = 'gray')
  tr = np.ones((dcp.shape[0], dcp.shape[1]))
  for i in range(dcp.shape[0]):
    for j in range(dcp.shape[1]):
      tr[i][j] = 1 - dcp[i][j]
  #ax[1].imshow(tr, cmap = 'gray')
  if retMean == False:
    return tr
  elif retMean == True:
    return np.mean(tr)

In [0]:
def entropy(image):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  hist = cv2.calcHist( [image.astype('float32')],
              [0], 
              None,
               [256], 
              [0,256] )
  h_norm = 0
  h_norm = cv2.normalize(hist, h_norm)
  ent = 0
  for p in h_norm:
    try:
      ent += p*math.log(p, 2)
    except:
      pass
    
  return (-1 * ent[0])

In [0]:
def contrast(image):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  s = 0
  ss = 0
  mean = np.mean(image)
  x = (-1 * mean * (np.ones(image.shape)))
  ss = np.square(np.add(image, x))
  cont = np.sqrt(np.sum(ss)/float(image.size))
  return cont

In [0]:
#Reading images from all folders : 
images = []




In [26]:
#Enhancing JPEG timestamp and cpcb timestamp compatibility
mint=[]
for name in imglenovo:
  #print(name)
  mint.append(int(name[-2:]))

print(jpg_names)
print(mint)
new_mint=[]
for j in mint: 
  if(j<7.5):
    new_mint.append('00')
  elif(j<15 or j<22.5):
    new_mint.append('15')
  elif(j<30 or j<37.5):
    new_mint.append('30')
  elif(j<45 or j<52.5):
    new_mint.append('45')
  else:
    new_mint.append('00')

print(len(new_mint))


#compat has original jpeg timestamp corresponding to the new timestamp 
compat={}
k=0
for name in jpg_names:
  compat[name]=str(name[:-2])+str(new_mint[k])
  k+=1 
  
print(compat)

ValueError: ignored